In [1]:
import pandas as pd
import os

from datetime import datetime

In [2]:
form = "%d/%m/%Y %H:%M:%S"

In [3]:
replacement = {"curva_direita_agressiva": "Aggressive right turn",
               "curva_esquerda_agressiva": "Aggressive left turn",
               "evento_nao_agressivo": "Non-aggressive event",
               "troca_faixa_direita_agressiva": "Aggressive right lane change",
               "aceleracao_agressiva": "Aggressive acceleration",
               "freada_agressiva": "Aggressive breaking",
               "troca_faixa_esquerda_agressiva": "Aggressive left lane change",
               "No label": "No label"}

In [4]:
def replace_event(row):
    return replacement[row['event']]

In [5]:
def make_labeled_data(folder_num):
    data_label = pd.read_csv(os.path.join('data', 'data_init', str(folder_num), 'groundTruth.csv'))
    data_acc = pd.read_csv(os.path.join('data', 'data_init', str(folder_num), 'aceleracaoLinear_terra.csv'))
    data_gyro = pd.read_csv(os.path.join('data', 'data_init', str(folder_num), 'giroscopio_terra.csv'))
    
    data = data_acc.copy()
    data.columns = ["timestamp", "uptimeNanos", "x_accelerometer", "y_accelerometer", "z_accelerometer"]
    data['x_gyroscope'] = data_gyro["x"]
    data['y_gyroscope'] = data_gyro["y"]
    data['z_gyroscope'] = data_gyro["z"]
    init = datetime.strptime(data.loc[0]['timestamp'], form)
    
    def change_timestamp(row):
        return (datetime.strptime(row['timestamp'], form) - init).seconds
    
    data['time_duration'] = data.apply(change_timestamp, axis=1)
    
    for index, row in data_label.iterrows():
        start = row[' inicio']
        finish = row[' fim']
        data.loc[((data['time_duration'] >= start) & (data['time_duration'] < finish)), 'event'] = row['evento']
    
    # Uncomment for saving only labeled data
    data = data.dropna(subset=["event"])
    data['event'] = data['event'].fillna("No label")
    data['event'] = data.apply(replace_event, axis=1)
    
    return data

In [6]:
data = make_labeled_data(16)
data.to_csv("data/data_accelerometer_gyroscope_16.csv", index=False)
# data.to_csv("data/data_accelerometer_gyroscope_16_all.csv", index=False)
data.shape

(2856, 10)

In [7]:
data = make_labeled_data(17)
data.to_csv("data/data_accelerometer_gyroscope_17.csv", index=False)
# data.to_csv("data/data_accelerometer_gyroscope_17_all.csv", index=False)
data.shape

(2241, 10)

In [8]:
data = make_labeled_data(20)
data.to_csv("data/data_accelerometer_gyroscope_20.csv", index=False)
# data.to_csv("data/data_accelerometer_gyroscope_20_all.csv", index=False)
data.shape

(2903, 10)

In [9]:
data = make_labeled_data(21)
data.to_csv("data/data_accelerometer_gyroscope_21.csv", index=False)
# data.to_csv("data/data_accelerometer_gyroscope_21_all.csv", index=False)
data.shape

(3161, 10)